In [ ]:
!pip install --upgrade openai wikipedia-api serpapi requests gtts onnx onnxruntime huggingface_hub streamlit amadeus faiss-cpu
!pip install --upgrade langchain-openai
!pip install --upgrade google-search-results

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=f86068c39e9665db55fb233e956ba

In [ ]:
%%writefile req.txt
streamlit
openai
wikipedia-api
faiss-cpu
transformers
sentence-transformers
onnx
onnxruntime
numpy
pillow
moviepy
gtts
requests
langchain
langchain-openai
amadeus
graphviz
seaborn
scikit-learn
matplotlib
plotly
gradio
diffusers
accelerate

Writing req.txt


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
# from moviepy.editor import AudioClip

In [ ]:
from serpapi import GoogleSearch
print("✅ GoogleSearch is imported successfully!")


✅ GoogleSearch is imported successfully!


In [ ]:
import openai
import wikipediaapi
import serpapi
import requests
import gtts
import onnx
import onnxruntime
import huggingface_hub
import streamlit
import amadeus
import faiss

print("✅ All libraries are installed successfully!")


✅ All libraries are installed successfully!


In [ ]:
# Create necessary files
!touch app.py travel_story.py config.py utils.py


In [ ]:
# from config import OPENAI_API_KEY

# print(OPENAI_API_KEY)


# **Write utils.py (Helper Functions for API Calls)**

In [ ]:
%%writefile utils.py

import requests
from config import OPENAI_API_KEY, GOOGLE_MAPS_API_KEY, WEATHER_API_KEY, AMADEUS_API_KEY, AMADEUS_API_SECRET, llm
from amadeus import Client, ResponseError
import os
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_openai import OpenAIEmbeddings
import re
import faiss
import numpy as np
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore  # ✅ Required for FAISS Storage
from langchain.schema import Document

amadeus = Client(client_id=AMADEUS_API_KEY, client_secret=AMADEUS_API_SECRET)

# ✅ Initialize OpenAI Embedding Model
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ FAISS Setup
dimension = 1536  # Ensure embedding dimensions match OpenAI embeddings
faiss_index = faiss.IndexFlatL2(dimension)
docstore = InMemoryDocstore({})
index_to_docstore_id = {}

vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)

# ✅ Fetch Travel Data
def fetch_travel_data(destination):
    """Retrieve real-time travel-related information from APIs."""

    print(f"Fetching travel data for {destination}...")

    # ✅ Wikipedia Data
    wiki_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{destination.replace(' ', '_')}"
    wiki_response = requests.get(wiki_url)
    wiki_data = wiki_response.json().get("extract", "No data available.") if wiki_response.status_code == 200 else "No data available."

    # ✅ Google Places Data
    google_places_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {"query": f"top attractions in {destination}", "key": GOOGLE_MAPS_API_KEY}
    google_response = requests.get(google_places_url, params=params)

    places_info = "No places found."
    if google_response.status_code == 200:
        places = google_response.json().get("results", [])
        places_info = "\n".join([f"{p['name']} - {p.get('formatted_address', 'No address')}" for p in places[:5]])

    return f"{wiki_data}\n\nTop Attractions:\n{places_info}"

# ✅ Update FAISS Index
def update_faiss_index(destination):
    """Dynamically update FAISS index and store documents properly."""

    print(f"🔄 Updating FAISS index for {destination}...")
    travel_data = fetch_travel_data(destination)

    # ✅ Generate embeddings dynamically
    doc_embedding = embedding_model.embed_documents([travel_data])
    embedding_dim = len(doc_embedding[0])  # ✅ Detect embedding dimension dynamically

    global faiss_index, vector_store, docstore, index_to_docstore_id

    if faiss_index.is_trained and faiss_index.ntotal > 0:
        existing_dim = faiss_index.d
        if existing_dim != embedding_dim:
            print(f"⚠️ FAISS dimension mismatch! Reinitializing FAISS to {embedding_dim} dimensions.")
            faiss_index = faiss.IndexFlatL2(embedding_dim)
            docstore = InMemoryDocstore({})
            index_to_docstore_id = {}

    else:
        print(f"✅ Initializing FAISS with {embedding_dim} dimensions.")
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        docstore = InMemoryDocstore({})
        index_to_docstore_id = {}

    # ✅ Add new embeddings to FAISS
    faiss_index.add(np.array(doc_embedding))

    # ✅ Store document properly in docstore
    doc_id = str(faiss_index.ntotal - 1)
    index_to_docstore_id[faiss_index.ntotal - 1] = doc_id
    docstore._dict[doc_id] = Document(page_content=travel_data, metadata={"doc_id": doc_id})

    # ✅ Save FAISS Index
    vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)
    vector_store.save_local("faiss_travel_index")

    print(f"✅ FAISS Index successfully updated with travel data for {destination}")

# ✅ Retrieve Data using FAISS
def retrieve_relevant_docs(query):
    """Retrieve relevant travel data from FAISS dynamically."""

    print(f"🔍 Retrieving relevant documents for query: {query}")

    # ✅ Load FAISS index safely
    vector_store = FAISS.load_local(
        "faiss_travel_index",
        OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
        allow_dangerous_deserialization=True
    )

    retrieved_docs = vector_store.similarity_search_with_score(query, k=2)

    if not retrieved_docs:
        print("❌ No relevant documents found.")
        return "No relevant documents found for your query."

    print(f"✅ Relevant Documents Found: {retrieved_docs}")
    return " ".join([doc.page_content for doc, _ in retrieved_docs])

# ✅ Generate AI Travel Plan using RAG
def generate_travel_plan_rag(origin, destination, start_date, end_date, purpose):
    """Use RAG to generate a detailed travel plan dynamically."""

    print("🔄 Generating AI Travel Plan using RAG...")

    # ✅ Retrieve Travel Information from FAISS
    context_info = retrieve_relevant_docs(f"Best travel itinerary for {destination}")

    prompt = f"""
    Create a detailed travel itinerary for {destination} from {origin} ({start_date} - {end_date}).
    Purpose: {purpose}

    Additional Travel Information:
    {context_info}
    """

    response = llm.invoke(prompt)
    return response.content if hasattr(response, 'content') else str(response)

# ✅ Generate AI Travel Story using RAG
def generate_travel_story_rag(origin, destination, start_date, end_date, purpose):
    """Use RAG to generate a travel story dynamically."""

    print("🔄 Generating AI Travel Story using RAG...")

    # ✅ Retrieve Travel Information from FAISS
    context_info = retrieve_relevant_docs(f"Best places to visit in {destination} for {purpose}")

    prompt = f"""
    Create a compelling travel story about visiting {destination} from {origin} ({start_date} - {end_date}).
    Purpose: {purpose}

    Additional Travel Information:
    {context_info}
    """

    response = llm.invoke(prompt)
    return response.content if hasattr(response, 'content') else str(response)


# ✅ Function to fetch latitude & longitude
def get_lat_lng(location):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": location, "key": GOOGLE_MAPS_API_KEY}
    response = requests.get(url, params=params).json()
    if "results" in response and response["results"]:
        location_data = response["results"][0]["geometry"]["location"]
        return location_data["lat"], location_data["lng"]
    return None, None

# ✅ Function to fetch tourist attractions
def fetch_tourist_attractions(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 10000, "type": "tourist_attraction", "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{t['name']} ({t.get('rating', 'No rating')}⭐)" for t in response["results"][:top_n]]
    return "No tourist attractions found."

# ✅ Function to fetch restaurants
def fetch_restaurants(location, purpose, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    keyword = {
        "Leisure": "casual dining",
        "Business": "fine dining",
        "Family": "family-friendly",
        "Adventure": "unique cuisine",
        "Romantic": "romantic restaurant"
    }.get(purpose, "restaurant")

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "restaurant", "keyword": keyword, "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{r['name']} ({r.get('rating', 'No rating')}⭐)" for r in response["results"][:top_n]]
    return "No restaurants found."

# ✅ Function to fetch real-time weather details
def fetch_weather(city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": WEATHER_API_KEY, "units": "metric"}
    response = requests.get(url, params=params).json()
    if "weather" in response and "main" in response:
        return f"{response['weather'][0]['description'].capitalize()}, {response['main']['temp']}°C"
    return "Weather data not available."

def get_airport_code(city_name):
    """
    Convert a city name to an airport code using the Amadeus API.
    """
    try:
        response = amadeus.reference_data.locations.get(
            keyword=city_name,
            subType='AIRPORT'
        )
        # Extract the airport code from the response
        for location in response.data:
            if location['subType'] == 'AIRPORT':
                return location['iataCode']
        return None
    except ResponseError as error:
        print(f"Error fetching airport code for {city_name}: {error}")
        return None


# Helper Function to get full airline names from its codes using OpenAI
def get_airline_full_name(airline_code):
    prompt = f"Please provide only the full name for the airline '{airline_code}'."
    response = llm([HumanMessage(content=prompt)])
    return response.content.strip() if response else airline_code  # Return the code if response is empty

def format_duration(iso_duration):
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?', iso_duration)
    hours = match.group(1) if match.group(1) else "0"
    minutes = match.group(2) if match.group(2) else "0"
    return f"{int(hours)} hours {int(minutes)} minutes"

# Function to fetch Flights information
def fetch_flight_details(origin, destination, start_data, return_date=None, max_price=None, airline_name =None):
    try:
        origin_code = get_airport_code(origin)
        destination_code = get_airport_code(destination)
        # Set a high default max_price if not provided
        max_price = max_price if max_price else 20000
        params = {
            "originLocationCode": origin_code,
            "destinationLocationCode": destination_code,
            "departureDate": start_data,
            "adults": 1,
            "maxPrice": max_price
        }

        if return_date:
            params["returnDate"] = return_date  # Include return date for round-trip flights

        # Fetch flights from Amadeus API
        response = amadeus.shopping.flight_offers_search.get(**params)
        flights = response.data

        if flights:
            result = []
            for flight in flights[:5]:  # Limit to top 5 results
                if float(flight['price']['total']) <= max_price:
                    # Outbound flight details
                    segments = flight['itineraries'][0]['segments']
                    airline_code = segments[0]['carrierCode']
                    airline = get_airline_full_name(airline_code)  # Get full airline name
                    # Only add flights that match the specified airline, if provided
                    if airline_name and airline and airline.lower() not in airline_name.lower():
                        continue
                    departure_time = segments[0]['departure']['at']
                    arrival_time = segments[-1]['arrival']['at']
                    flight_duration = format_duration(flight['itineraries'][0]['duration'])

                    # Only include return details if a return date is provided
                    if return_date and len(flight['itineraries']) > 1:
                        return_segments = flight['itineraries'][1]['segments']
                        return_departure_time = return_segments[0]['departure']['at']
                        return_arrival_time = return_segments[-1]['arrival']['at']
                        return_duration = format_duration(flight['itineraries'][1]['duration'])
                        return_info = (
                            f"\nReturn Departure: {return_departure_time}\n"
                            f"Return Arrival: {return_arrival_time}\n"
                            f"Return Duration: {return_duration}\n"
                        )
                    else:
                        return_info = ""

                    # Append both outbound and return information (if available) to results
                    result.append(
                        f"Airline: {airline}\nPrice: ${flight['price']['total']}\n"
                        f"Departure: {departure_time}\nArrival: {arrival_time}\n"
                        f"Duration: {flight_duration}{return_info}"
                        "\n----------------------------------------"
                    )
            return "\n\n".join(result) if result else "No flights found within the budget."
        return "No flights found."
    except ResponseError as error:
        return f"An error occurred: {error.response.result}"

# ✅ Function to fetch hotels
def fetch_hotels(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "lodging", "key": GOOGLE_MAPS_API_KEY}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return [f"{h['name']} ({h.get('rating', 'No rating')}⭐)" for h in response["results"][:top_n]]
    return "No hotels found."



Overwriting utils.py


# **Write travel_story.py (AI-Powered Travel Story & Video Generation)**

In [ ]:
%%writefile travel_story.py

import openai
import requests
import wikipediaapi
import io
import time
import onnx
import numpy as np
from PIL import Image
# from moviepy.editor import *  # For video editing
from gtts import gTTS  # For text-to-speech
from onnxruntime import InferenceSession
from transformers import AutoModelForCausalLM, AutoTokenizer
from config import OPENAI_API_KEY # Import the latest API key from config.py
from config import llm, amadeus
import openai
import faiss
from moviepy.editor import AudioFileClip, ImageClip, concatenate_videoclips
from utils import update_faiss_index , generate_travel_story_rag, generate_travel_plan_rag, retrieve_relevant_docs
from io import BytesIO

# ✅ Ensure OpenAI API is using the latest key
openai.api_key = OPENAI_API_KEY  # This ensures the latest key is used every time

# ✅ Initialize OpenAI Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="MyTravelApp/1.0", language="en")

# ✅ Function to fetch travel data from Wikipedia
def get_wikipedia_summary(place):
    page = wiki.page(place)
    return page.summary[:500] if page.exists() else "No Wikipedia summary found."

def generate_travel_story(origin, destination, purpose, start_date, end_date):
    # wikipedia_info = get_wikipedia_summary(destination)
    update_faiss_index(destination)
    wikipedia_info = generate_travel_story_rag(origin, destination, start_date, end_date, purpose)

    purpose_templates = {
      "leisure": f"You are about to embark on a relaxing leisure trip starting from {origin} to {destination} from {start_date} to {end_date}. Describe your journey, including your departure experience, flight details, and how you arrive at {destination}. Highlight the famous landmarks, scenic parks, and peaceful experiences during your visit.",

      "food": f"As a food lover, you're traveling from {origin} to {destination} from {start_date} to {end_date} to explore its vibrant culinary scene. Describe the unique food experiences from the departure airport to your destination, including delicious street food, high-end restaurants, and bustling food markets in {destination}. Mention iconic cafés and dishes travelers should not miss.",

      "adventure": f"You're departing from {origin} to {destination} for an adrenaline-filled adventure from {start_date} to {end_date}. Describe your travel experience, flight, and arrival at {destination}. Highlight the thrilling activities such as hiking, surfing, skydiving, and other outdoor experiences that make this trip exhilarating.",

      "business": f"You're traveling from {origin} to {destination} for a business trip from {start_date} to {end_date}. Describe your departure from {origin}, your flight experience, and how you arrive at {destination}. Detail your meetings, networking events, and the city's corporate atmosphere. Also, mention after-hours dining or sightseeing to balance work and leisure.",

      "romantic": f"You're setting off from {origin} to {destination} for a romantic getaway from {start_date} to {end_date}. Describe the journey from {origin}, including your travel experience and how you and your partner arrive at {destination}. Highlight intimate dinners, scenic walks, breathtaking sunset views, and special moments shared during this trip.",

      "spiritual": f"You're traveling from {origin} to {destination} for a spiritual retreat from {start_date} to {end_date}. Describe your departure experience from {origin}, flight details, and arrival at {destination}. Mention meditation spots, temples, churches, and peaceful landscapes that provide a sense of tranquility and reflection.",

      "family": f"You're taking a family trip from {origin} to {destination} from {start_date} to {end_date}, creating memorable bonding moments. Describe your journey, including how you and your family prepare for the trip, your flight experience, and your arrival at {destination}. Highlight amusement parks, kid-friendly attractions, and activities that make this a joyful and unforgettable experience for everyone."
    }


    purpose_prompt = purpose_templates.get(purpose, purpose_templates["leisure"])

    full_prompt = f"""
    {purpose_prompt}

    Be immersive, engaging, and detailed. Use vivid descriptions and include unique aspects of {destination}.

    Wikipedia Summary: {wikipedia_info}

    Travel Story:
    """

    response = llm.invoke(full_prompt)
    return response.content if hasattr(response, 'content') else str(response)

# ✅ Function to generate a travel plan
def generate_travel_plan(origin, destination, start_date, end_date, purpose):
    update_faiss_index(destination)
    travel_plan_context = generate_travel_plan_rag(origin, destination, start_date, end_date, purpose)
    prompt = f"""
    Generate a detailed travel itinerary for a trip starting from {origin} to {destination} from {start_date} to {end_date} for {purpose}. Use {travel_plan_context} as a reference.

    Include the following details:
    - **Departure details** from {origin}, including flight or transportation options.
    - **Arrival experience** in {destination} and first impressions.
    - **Accommodation recommendations** suitable for {purpose}.
    - **Top attractions** in {destination} that match {purpose}.
    - **Food and dining recommendations**, including famous restaurants.
    - **Local transportation options** to navigate within {destination}.
    - **Return trip details** from {destination} back to {origin} (if applicable).

    Ensure the itinerary is engaging and structured as a day-by-day plan.

    Travel Itinerary:
    """
    response = llm.invoke(prompt)
    return response.content if hasattr(response, 'content') else str(response)

# ✅ Function to extract day-wise highlights
def extract_daywise_highlights(travel_plan):
    days = travel_plan.split("Day ")[1:]
    daywise_highlights = {}
    for day in days:
        lines = day.split("\n")
        day_number = lines[0].strip()
        activities = ". ".join([line.strip() for line in lines[1:] if line.strip()])
        daywise_highlights[day_number] = activities
    return daywise_highlights

# ✅ Function to generate travel images per day's activity
def generate_travel_images(daywise_highlights, destination):
    image_urls = {}
    for day, activities in daywise_highlights.items():
        prompt = f"Generate an ultra-HD image representing {activities} in {destination}."
        response = client.images.generate(
            model="dall-e-3", prompt=prompt, n=1, size="1024x1024"
        )
        image_urls[day] = response.data[0].url
    return image_urls

# ✅ Function to generate a voice-over for the itinerary
def generate_voiceover(travel_plan, output_audio="travel_narration.mp3"):
    tts = gTTS(text=travel_plan, lang="en", slow=False)
    tts.save(output_audio)
    return output_audio

def create_travel_video(image_urls, narration_audio, output_video="travel_story.mp4"):
    # Load the narration audio
    audio_clip = AudioFileClip(narration_audio)
    total_audio_duration = audio_clip.duration
    num_days = len(image_urls)
    image_duration = total_audio_duration / num_days  # Divide equally among days

    image_clips = []
    for day, url in image_urls.items():
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image_path = f"travel_image_{day}.jpg"
            image.save(image_path)

            # Create an ImageClip with the specified duration
            clip = ImageClip(image_path, duration=image_duration).set_fps(24)
            clip = clip.resize(lambda t: 1 + 0.01 * t)  # Slow zoom-in effect
            image_clips.append(clip)
        else:
            print(f"Unable to fetch image for Day {day}. Status code: {response.status_code}")

    # Concatenate all image clips into a single video
    video_clip = concatenate_videoclips(image_clips, method="compose")
    video_clip = video_clip.set_audio(audio_clip)

    # Write the video file
    video_clip.write_videofile(output_video, codec="libx264", fps=24, audio_codec="aac")
    print("🎬 Personalized travel video created successfully!")
    return output_video

# ✅ Execution for Testing
if __name__ == "__main__":
    origin = "New York"
    destination = "Hyderabad"
    purpose = "Family"
    start_date = "2025-04-15"
    end_date = "2025-04-20"

    print("📅 Generating Travel Plan...")
    travel_plan = generate_travel_plan(origin, destination, start_date, end_date, purpose)
    print(f"📝 Travel Plan:\n{travel_plan}")

    # ✅ Extract day-wise activities
    daywise_highlights = extract_daywise_highlights(travel_plan)

    print("🔄 Fetching data and generating travel story...")
    travel_story_text = generate_travel_story(origin, destination, purpose, start_date, end_date)
    print(f"📖 Travel Story:\n{travel_story_text}")

    print("🖼 Generating Travel Images...")
    image_urls = generate_travel_images(daywise_highlights, destination)
    print(f"Generated Images: {image_urls}")

    print("🎤 Generating voiceover...")
    narration_file = generate_voiceover(travel_story_text)

    print("🎥 Creating travel video...")
    travel_video = create_travel_video(image_urls, narration_file)
    print(f"✅ Travel video saved as {travel_video}")


Overwriting travel_story.py


In [ ]:
from travel_story import generate_travel_story, generate_travel_plan

# travel_story_text = generate_travel_story("Hyderabad", "Paris", "Family", "2025-04-25", "2025-04-30")
travel_plan_text = generate_travel_plan("Hyderabad", "Paris", "2025-04-25", "2025-04-30", "Family")
print(travel_plan_text)

🔄 Updating FAISS index for Paris...
Fetching travel data for Paris...
✅ FAISS Index successfully updated with travel data for Paris
🔄 Generating AI Travel Plan using RAG...
🔍 Retrieving relevant documents for query: Best travel itinerary for Paris
✅ Relevant Documents Found: [(Document(metadata={'doc_id': '0'}, page_content="Paris is the capital and largest city of France. With an estimated population of 2,048,472 residents in January 2025 in an area of more than 105\xa0km2 (41\xa0sq\xa0mi), Paris is the fourth-most populous city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. Because of its leading role in the arts and sciences and its early adaptation of extensive street lighting, Paris became known as the City of Light in the 19th century.\n\nTop Attractions:\nGrand Palais - 75008 Paris, France\nArc de Triomph

# **Write app.py (Streamlit UI)**

In [ ]:
%%writefile app.py

import streamlit as st
from travel_story import generate_travel_story, generate_voiceover, create_travel_video, generate_travel_plan, generate_travel_images, extract_daywise_highlights
from utils import fetch_weather, fetch_tourist_attractions, fetch_flight_details, fetch_restaurants, fetch_hotels
from config import llm  # Ensure llm is imported for LLM calls
import requests
from PIL import Image
from io import BytesIO
from moviepy.editor import AudioFileClip

# Set up Streamlit Page
st.set_page_config(page_title="✈️ AI Travel Planner", layout="wide")

st.title("✈️ AI Travel Planner 🏨")

# Sidebar Inputs
st.sidebar.title("Plan Your Trip 🗺")
origin = st.sidebar.text_input("Enter Origin City", "New York")
destination = st.sidebar.text_input("Enter Destination", "Paris")
start_date = st.sidebar.date_input("Start Date")
end_date = st.sidebar.date_input("End Date")
purpose = st.sidebar.selectbox("Purpose of Visit", ["Leisure", "Business", "Adventure", "Romantic", "Family"])

# Initialize session state for storing travel plan and conversation
if "travel_plan" not in st.session_state:
    st.session_state.travel_plan = None
    st.session_state.travel_details = {}

if "conversation" not in st.session_state:
    st.session_state.conversation = []

# Keep track of message processing state
if "message_to_process" not in st.session_state:
    st.session_state.message_to_process = None

# Generate Travel Plan & Fetch Details
if st.sidebar.button("Generate Travel Plan & Details", key="gen_plan_btn"):
    with st.spinner("🔄 Generating AI travel plan & fetching details..."):
        travel_plan_text = generate_travel_plan(origin, destination, start_date, end_date, purpose)
        weather_info = fetch_weather(destination)
        tourist_attractions = fetch_tourist_attractions(destination)
        restaurants = fetch_restaurants(destination, purpose)
        hotels = fetch_hotels(destination)
        flights = fetch_flight_details(origin, destination, start_date, return_date=None, max_price=None, airline_name=None)

        # Store the travel plan in session state for the chatbot to use
        st.session_state.travel_plan = travel_plan_text
        st.session_state.travel_details = {
            "weather": weather_info,
            "attractions": tourist_attractions,
            "restaurants": restaurants,
            "hotels": hotels,
            "flights": flights
        }

    # Display the generated plan and details
    st.subheader("📅 Your AI-Generated Travel Plan")
    st.write(travel_plan_text)
    st.subheader(f"🌦 Weather Forecast in {destination}")
    st.write(weather_info)
    st.subheader(f"🏛 Top Attractions in {destination}")
    st.write(tourist_attractions)
    st.subheader(f"🍽 Best Restaurants in {destination}")
    st.write(restaurants)
    st.subheader(f"🏨 Recommended Hotels in {destination}")
    st.write(hotels)
    st.subheader(f"✈️ Flights from {origin} to {destination}")
    st.write(flights)

# Generate Travel Story & Voiceover
if st.sidebar.button("Generate Story & Voiceover", key="gen_story_btn"):
    with st.spinner("🔄 Generating AI travel story..."):
        travel_story_text = generate_travel_story(origin, destination, purpose, start_date, end_date)
        narration_audio = generate_voiceover(travel_story_text)

    st.subheader("📖 Your AI-Generated Travel Story")
    st.write(travel_story_text)
    st.subheader("🎤 AI Voiceover")
    st.audio(narration_audio)

# Generate Images & Video
if st.sidebar.button("Generate Images & Video", key="gen_media_btn"):
    with st.spinner("🖼 Generating Travel Images..."):
        travel_plan_text = generate_travel_plan(origin, destination, start_date, end_date, purpose)
        daywise_highlights = extract_daywise_highlights(travel_plan_text)
        travel_images = generate_travel_images(daywise_highlights, destination)

    st.subheader("🖼 View Destination Images")
    if travel_images:
        for day, image_url in travel_images.items():
            try:
                response = requests.get(image_url)
                if response.status_code == 200:
                    image = Image.open(BytesIO(response.content))
                    st.image(image, caption=f"Day {day}: A view of {destination}", use_column_width=True)
                else:
                    st.warning(f"❌ Unable to fetch image for Day {day}. Try again later.")
            except Exception as e:
                st.error(f"Error fetching image for Day {day}: {e}")
    else:
        st.warning("❌ No images generated.")

    with st.spinner("🎥 Creating AI Travel Video..."):
        travel_plan_text = generate_travel_plan(origin, destination, start_date, end_date, purpose)
        daywise_highlights = extract_daywise_highlights(travel_plan_text)
        travel_images = generate_travel_images(daywise_highlights, destination)
        narration_audio = generate_voiceover(travel_plan_text)
        travel_video = create_travel_video(travel_images, narration_audio)

    st.subheader("🎥 AI-Generated Travel Video")
    if travel_video:
        st.video(travel_video)
    else:
        st.warning("❌ Video generation failed.")

st.success("✅ AI Travel Planner is ready!")

# =======================
# Interactive Chat Section
# =======================

st.markdown("---")
st.header("🤖 Chat with Your Travel Planner")
st.write("Ask questions about your trip or request modifications to your plan.")


def update_message_to_process():
# First, check if there's a message to process from the previous run
    if st.session_state.message_to_process:
        with st.spinner("Processing your message..."):
            # Get the message to process
            message = st.session_state.message_to_process

            # Add user message to conversation
            st.session_state.conversation.append({"role": "user", "content": message})

            # Build context for the LLM based on existing travel information
            context = f"Origin: {origin}, Destination: {destination}, Dates: {start_date} to {end_date}, Purpose: {purpose}"

            # Create a comprehensive system prompt that includes the base travel plan
            travel_plan = st.session_state.travel_plan if st.session_state.travel_plan else "No travel plan has been generated yet."
            travel_details = st.session_state.travel_details

            system_prompt = f"""You are an AI travel assistant helping a user with their trip.

            TRIP DETAILS:
            - Origin: {origin}
            - Destination: {destination}
            - Dates: {start_date} to {end_date}
            - Purpose: {purpose}

            BASE TRAVEL PLAN:
            {travel_plan}

            ADDITIONAL INFORMATION:
            - Weather: {travel_details.get('weather', 'Not available')}
            - Top Attractions: {travel_details.get('attractions', 'Not available')}
            - Restaurants: {travel_details.get('restaurants', 'Not available')}
            - Hotels: {travel_details.get('hotels', 'Not available')}
            - Flights: {travel_details.get('flights', 'Not available')}

            INSTRUCTIONS:
            - Remember all details about the user's trip when answering questions
            - Be concise but informative in your responses
            - If the user asks about information not in the plan, respond with relevant suggestions
            - If the user wants to modify their plan, acknowledge this and explain how the modification fits with the overall trip
            """

            # Create messages array for the LLM
            messages = [{"role": "system", "content": system_prompt}]

            # Add conversation history to messages
            for msg in st.session_state.conversation:
                if msg["role"] != "system":  # Avoid duplicate system messages
                    messages.append({"role": msg["role"], "content": msg["content"]})

            # Generate assistant response using the LLM
            try:
                # Using the llm in a way that works with both older and newer LLM interfaces
                if hasattr(llm, 'chat'):
                    # For newer LLM interfaces that use the chat method
                    response = llm.chat(messages)
                    assistant_response = response.content if hasattr(response, "content") else str(response)
                else:
                    # For older LLM interfaces or those using direct invoke
                    full_prompt = system_prompt + "\n\n" + "\n".join([f"{m['role']}: {m['content']}" for m in messages if m['role'] != "system"])
                    response = llm.invoke(full_prompt)
                    assistant_response = response.content if hasattr(response, "content") else str(response)

                # Add assistant response to conversation history
                st.session_state.conversation.append({"role": "assistant", "content": assistant_response})

            except Exception as e:
                st.error(f"Error generating response: {str(e)}")

            # Clear the message to process
            st.session_state.message_to_process = None

# Display the conversation history
st.subheader("💬 Your Conversation")
for message in st.session_state.conversation:
    if message["role"] == "user":
        st.markdown(f"**You:** {message['content']}")
    else:
        st.markdown(f"**Planner:** {message['content']}")

# # Input for user's message
# user_input = st.text_input("Your message:", key="chat_input")
# send_button = st.button("Send", key="send_btn")

# if send_button and user_input:
#     # Store the message to process in the next run
#     st.session_state.message_to_process = user_input
#     # # Clear the input
#     # st.session_state.chat_input = ""
#     update_message_to_process()

# Initialize the key in session state if it doesn't exist
if "user_input" not in st.session_state:
    st.session_state.user_input = ""

# Input for user's message - use a callback to handle submissions
def submit_message():
    if st.session_state.user_input.strip():
        st.session_state.message_to_process = st.session_state.user_input
        st.session_state.user_input = ""
        update_message_to_process()

# Create the text input with the callback
user_input = st.text_input(
    "Your message:",
    key="user_input",
    on_change=submit_message
)

# # Add a send button that also triggers the same callback
# if st.button("Send", key="send_btn"):
#     submit_message()


Overwriting app.py


In [ ]:
# from travel_story import generate_travel_story, generate_voiceover, create_travel_video, generate_travel_plan, generate_travel_images
# from utils import fetch_weather, fetch_tourist_attractions, fetch_flight_details, fetch_restaurants, fetch_hotels
# from config import llm


# class SessionState:
#     def __init__(self):
#         self.conversation = []

# # Initialize session state
# session_state = SessionState()

# travel_plan_text = generate_travel_plan("New York", "New Delhi", "2025-04-15", "2025-04-20", "Family")
# print(travel_plan_text)

# def chatbot_loop():
#     """
#     Simulates a chatbot interaction where users provide input,
#     and the chatbot refines the travel plan iteratively.
#     """
#     print("🤖 AI Travel Planner Chatbot")
#     print("Type 'exit' to stop the chat.\n")

#     while True:
#         user_input = input("You: ")
#         if user_input.lower() == "exit":
#             print("\nChat ended.")
#             break

#         # Append user message to conversation history
#         session_state.conversation.append({"role": "user", "content": user_input})

#         # Construct conversation context
#         conversation_context = "\n".join(
#             [f"{msg['role']}: {msg['content']}" for msg in session_state.conversation]
#         )
#         # print(conversation_context)

#         prompt = (
#             f"Below is the base travel plan:\n{travel_plan_text}\n\n"
#             f"And here is the conversation with the user:\n{conversation_context}\n\n"
#             "Based on the above, please respond to the user answering his queries."
#         )

#         # Mock LLM call (replace with actual LLM API call in production)
#         ai_response = llm.invoke(prompt)

#         # Append AI response to conversation history
#         session_state.conversation.append({"role": "assistant", "content": ai_response})

#         # Print the response
#         print(f"Planner: {ai_response}\n")


# # Run the chatbot loop
# chatbot_loop()


In [ ]:
%%writefile verify_imports.py

import os

# ✅ Check if all necessary files exist
required_files = ["config.py", "utils.py", "travel_story.py", "app.py"]
missing_files = [file for file in required_files if not os.path.exists(file)]

if missing_files:
    print(f"❌ ERROR: Missing files: {missing_files}. Ensure all required files are present.")
else:
    print("✅ All necessary files exist.")

# ✅ Verify `config.py` imports
try:
    from config import google_maps_api_key, serpapi_key, WEATHER_API_KEY
    print("✅ Successfully imported API keys from config.py")
    print(f"Google Maps API Key: {google_maps_api_key[:5]}******")
    print(f"SerpAPI Key: {serpapi_key[:5]}******")
    print(f"Weather API Key: {weather_api_key[:5]}******")
except ModuleNotFoundError:
    print("❌ ERROR: 'config.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import variables from 'config.py'.")

# ✅ Verify `utils.py` imports
try:
    from utils import get_lat_lng, fetch_restaurants, fetch_weather
    print("✅ Successfully imported functions from utils.py")
    print(f"get_lat_lng function exists: {callable(get_lat_lng)}")
    print(f"fetch_restaurants function exists: {callable(fetch_restaurants)}")
    print(f"fetch_weather function exists: {callable(fetch_weather)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'utils.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'utils.py'.")

# ✅ Verify `purpose.py` imports
try:
    from travel_story import generate_travel_story, generate_voiceover, create_travel_video
    print("✅ Successfully imported functions from purpose.py")
    print(f"generate_travel_story function exists: {callable(generate_travel_story)}")
    print(f"generate_voiceover function exists: {callable(generate_voiceover)}")
    print(f"create_travel_video function exists: {callable(create_travel_video)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'purpose.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'purpose.py'.")

# ✅ Verify `app.py` existence
if os.path.exists("app.py"):
    print("✅ 'app.py' exists and is ready to run.")
else:
    print("❌ ERROR: 'app.py' is missing.")


Overwriting verify_imports.py


In [ ]:
!pip install pyngrok

from pyngrok import ngrok
!kill $(pgrep -f ngrok)

# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &
# Set up ngrok
!ngrok authtoken 2sbDJyD6PkmqFpPwYo0r7FDObwZ_7cGS5ZDcbPxhQRJ6Swtzg

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

# Run the Streamlit app
!streamlit run app.py &

^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://a7d7-34-91-103-28.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.91.103.28:8502



In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

from travel_story import generate_travel_story, generate_voiceover

travel_story_text = generate_travel_story("New York", "New Delhi", "Family", "2025-04-15", "2025-04-20")
narration_audio = generate_voiceover(travel_story_text)

y, sr = librosa.load(narration_audio, sr=None)

# Plot the waveform
plt.figure(figsize=(14, 5))
librosa.display.waveshow(y, sr=sr, alpha=0.6)
plt.title("Voiceover Waveform")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
# Let's say you want to zoom in on the first 10 seconds
zoom_start = 0
zoom_end = int(10 * sr)  # 10 seconds worth of samples
y_zoom = y[zoom_start:zoom_end]

plt.figure(figsize=(14, 5))
librosa.display.waveshow(y_zoom, sr=sr, alpha=0.6)
plt.title("Zoomed-in Waveform (First 10 Seconds)")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
import numpy as np

# Compute the Short-Time Fourier Transform (STFT)
D = np.abs(librosa.stft(y))
# Convert amplitude to decibels
DB = librosa.amplitude_to_db(D, ref=np.max)

plt.figure(figsize=(14, 5))
librosa.display.specshow(DB, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram")
plt.xlabel("Time (seconds)")
plt.ylabel("Frequency (Hz)")
plt.show()

In [ ]:
# Compute RMS energy
rms = librosa.feature.rms(y=y)[0]
# Convert frame indices to time
frames = range(len(rms))
t = librosa.frames_to_time(frames, sr=sr)

plt.figure(figsize=(14, 5))

# Plot the raw waveform with reduced opacity
librosa.display.waveshow(y, sr=sr, alpha=0.4, label='Waveform')

# Overlay the RMS energy
plt.plot(t, rms, color='r', label='RMS Energy')

plt.title("Waveform with RMS Energy")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude / RMS")
plt.legend()
plt.show()